## Import libraries

In [ ]:
# physics engine
import pybullet as pb
import pybullet_data

# Franka simulator
from franka_env import FrankaEnv

# Python libraries
import numpy as np
import time
import math
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat

## Parameters

In [ ]:
simFranka = False # whether to open Graphical interface to animate Franka motion
np.set_printoptions(precision=3)
nStates = 10
accuracy_invKin = 0.000001

## User Functions

In [ ]:
def quat_to_euler(q):
    """ 
    Function that converts quaternions to Euler angles
    
    Inputs
    ------
    q : quaternions
    
    Outputs
    -------
    
    phi, theta, psi : Euler angles
    """
    pi = math.pi
    q0, q1, q2, q3 = q[3], q[0], q[1], q[2]
    test = np.dot(q0,q2) - np.dot(q1,q3)
    if (test > 0.4999): 
        phi = math.atan2(q1*q2 - q0*q3, q1*q3 + q0*q2)
        theta = pi/2
        psi= 0
    elif (test < -0.4999):
        phi = math.atan2(- q1*q2 + q0*q3, - q1*q3 - q0*q2)
        theta = -pi/2
        psi = 0
    else:
        sqx = q1*q1
        sqy = q2*q2
        sqz = q3*q3
        
        psi = math.atan2(2*q2*q1+2*q0*q3 , 1 - 2*sqy - 2*sqz) # Yaw
        theta = math.asin(2*test) # Pitch
        phi = math.atan2(2*q2*q3+2*q0*q1 , 1 - 2*sqx - 2*sqy) # Roll
    return phi, theta, psi

def Psi_x(x): 
    return x

def Run_Franka(Steps, state_desired, LQR_gains, x0 = None, y0 = None, z0 = None):
    """
    Function that simulates forward the dynamics of Franka using LQR control, 
    given a desired trajectory and LQR gains
    
    Inputs
    ------
    Steps : Number of steps to propagate discrete dynamics into the future
    state_desired: desired trajectory to track (used for the applied control)
    LQR_gains: LQR gains used together with the current state and desired trajectory 
            to compute control response
            
    Outputs
    -------
    state_traj : state trajectory of the controlled system
    control_traj : control trajectory that was applied to the system
    """
    state = env.reset()
    
    # Set state to desired state to help initiation
    for i, jnt in enumerate(state_desired[0,3:9]):
            pb.resetJointState(env.robot, i, jnt)
    
    # set initial y,z states to given y0, z0
    if x0:
        JointAngles_Fig8 = accurateCalculateInverseKinematics(env.robot, env.ee_id, [x0, y0, z0], accuracy_invKin, 10000)
        for i, jnt in enumerate(JointAngles_Fig8[0:6]):
            pb.resetJointState(env.robot, i, jnt)

    state = env.get_state()

    # Initialize states and controls trajectories
    state_traj, control_traj = np.empty((Steps+1,17)), np.empty((Steps, 7))
    state_traj[:], control_traj[:] = np.NaN,  np.NaN
    
    # Simulate dynamics forward
    for t in range(Steps):
        state = np.append(state[0:3], state[7:])
        state = Psi_x(state)
        state_traj[t, :] = state[:17]
        
        control = - np.dot(LQR_gains, (state - state_desired[t+1,:]))
        control_traj[t, :] = control
        state = env.step(control)
#         time.sleep(1.0/6000.)
#         time.sleep(1.0/600000.)
    
    state = np.append(state[0:3], state[7:])
    state = Psi_x(state)
    state_traj[t+1, :] = state[:17]
    return state_traj, control_traj

def accurateCalculateInverseKinematics(kukaId, endEffectorId, targetPos, threshold, maxIter):
    """
    Calculates the joint poses given the End Effector location using inverse kinematics
    Note: It changes the Franka configuration during the optimization to the desired configuration
    
    Input: 
    kukaId : Object that represents the Franka system
    endEffectorId : 
    targetPos : 
    threshold : accuracy threshold
    maxIter : maximum iterations to fine tune solution
    
    Output: 
    jointPoses: The angles of the 7 joints of the Franka 
    """
    numJoints = 7
    closeEnough = False
    iter = 0
    dist2 = 1e30
    while (not closeEnough and iter < maxIter):
        jointPoses = pb.calculateInverseKinematics(kukaId, endEffectorId, targetPos)
    
        for i in range(numJoints):
            pb.resetJointState(kukaId, i, jointPoses[i])
        ls = pb.getLinkState(kukaId, endEffectorId)
        newPos = ls[4]
        diff = [targetPos[0] - newPos[0], targetPos[1] - newPos[1], targetPos[2] - newPos[2]]
        dist2 = (diff[0] * diff[0] + diff[1] * diff[1] + diff[2] * diff[2])
        closeEnough = (dist2 < threshold)
        iter = iter + 1
    return jointPoses[:7]

def runLQRonFranka(steps, desired_target, LQR_gains, x0 = None, y0 = None, z0 = None, method = False):
    """
    Executes a simulation of specified number of steps applying LQR on Franka with the input LQR gains
        
    Inputs
    ------
    steps : number of steps to forward simulate the system
    desired_target : desired_trajectory to track
    LQR_gains : LQR gains used to compute control
    method : method to print on pdf label; if none is specified, no pdf is created
"""
    state_traj, controls_traj = Run_Franka(steps, desired_target, LQR_gains, x0, y0, z0)

    error = np.linalg.norm(state_traj[:,:nStates] - desired_target[:,:nStates])
    if method:
        fig = plt.plot(state_traj[:,1], state_traj[:,2], 'b-', linewidth = 1, markersize = 1)
        plt.plot(desired_target[:,1], desired_target[:,2], 'k--', linewidth = 1)
        plt.axis('equal')
        plt.title(method +': Error = {0:.2f}'.format(error), fontdict=None, loc='center', pad=None)
    return state_traj, controls_traj, error

def desiredStates_from_EndEffector(xyzEndEffector):
    """
    This function takes in the x,y,z coordinates of the end effector (EE) of Franka 
    and returns the closest (locally, using Inverse Kinematics) desired configuration
    to achieve that EE position . 
    
    Inputs
    ------
    x_d, y_d, z_d: End effector coordinates
    
    Outputs
    desired_states: x, y, z, JointAngles, Joint Velocities
    """
    x_d, y_d, z_d = xyzEndEffector[0], xyzEndEffector[1], xyzEndEffector[2]
    jointAngles = np.asarray(accurateCalculateInverseKinematics(env.robot, env.ee_id, [x_d, y_d, z_d], accuracy_invKin, 10000))
    state_des = np.concatenate((xyzEndEffector, jointAngles, np.zeros(7)))

    return state_des

In [ ]:
# initialize Franka object
env = FrankaEnv(render = simFranka)

In [ ]:
env.reset()
T = 6 *10 # time horizon 
t = 1.6 + 0.02*np.linspace(0, T*5, T*50+1) # time steps
Steps = len(t)-1


a = 0.3 # scaling parameter of Fig. 8 area
x = 0.3*np.ones((len(t),1)) 
z = np.expand_dims(0.59 + 2* a * np.sin(t) * np.cos(t) / (1+np.sin(t)**2), axis = 1)
y = np.expand_dims(a * np.cos(t) / (1+np.sin(t)**2), axis = 1)
plt.plot(y,z)
plt.axis('equal')
plt.xlabel('y (m)', Fontsize = 14)
plt.ylabel('z (m)', Fontsize = 14)

In [ ]:
# Translate desired y-z coordinates to desired joint angles of the Franka robotic arm
JointAngles_Fig8 = np.empty((len(t),7))
JointAngles_Fig8[:] = np.NaN
for i in range(len(t)):
    JointAngles_Fig8[i,:] = accurateCalculateInverseKinematics(env.robot, env.ee_id, [x[i], y[i], z[i]], accuracy_invKin, 10000)
states_des = np.concatenate( (x, y, z, JointAngles_Fig8, np.zeros((len(y), 7))), axis = 1)
# states_des = np.concatenate((x,y,z), axis = 1)


## Load LQR gains

In [ ]:
LQR_gains = loadmat('LQR_gains.mat')['LQR']
LQR_SOC, LQR_LS, LQR_CG, LQR_WLS = LQR_gains['SUB'][0,0], LQR_gains['LS'][0,0], LQR_gains['CG'][0,0], LQR_gains['WLS'][0,0]

print(np.linalg.norm(LQR_CG), np.linalg.norm(LQR_SOC), np.linalg.norm(LQR_LS),  np.linalg.norm(LQR_WLS))
print(LQR_CG.shape)

# Monte Carlo on Initial Conditions

In [ ]:
for i in range(50):
    y0, z0 = y[0,0] + np.random.uniform(-0.1, 0.1, 1), z[0,0] + np.random.uniform(-0.1, 0.1, 1)
    
    state_traj_LS, controls_traj_LS, error_LS = runLQRonFranka(Steps, states_des, LQR_LS, x[0,0], y0, z0)
    state_traj_SOC, controls_traj_SOC, error_SOC = runLQRonFranka(Steps, states_des, LQR_SOC, x[0,0], y0, z0)
    state_traj_CG, controls_traj_CG, error_CG = runLQRonFranka(Steps, states_des, LQR_CG, x[0,0], y0, z0)
    state_traj_WLS, controls_traj_WLS, error_WLS = runLQRonFranka(Steps, states_des, LQR_WLS, x[0,0], y0, z0)
    savemat('FrankaFig8_SimData'+str(i)+'.mat', {'desired_states':states_des,  'states_SOC': state_traj_SOC, 'u_SOC' : controls_traj_SOC,
                                                 'states_LS': state_traj_LS, 'u_LS' : controls_traj_LS,
                                                 'states_WLS': state_traj_WLS, 'u_WLS' : controls_traj_WLS,
                                                 'states_CG': state_traj_CG, 'u_CG' : controls_traj_CG}) # save variables to Matlab file
    
